In [1]:
# From: Bayesian Models for Astrophysical Data, Cambridge Univ. Press
# (c) 2017,  Joseph M. Hilbe, Rafael S. de Souza and Emille E. O. Ishida 
# 
# you are kindly asked to include the complete citation if you used this 
# material in a publication
#
# Code 10.19 Bernoulli logit model, in Python using Stan, for assessing 
#            the relationship between Seyfert AGN activity and 
#            galactocentric distance
#
# Statistical Model: Bernoulli mixed model in Python using Stan
#
# Astronomy case: Relationship between Seyfert activity and 
#                 cluster centric distance - taken from 
#                 de Souza et al., 2016, MNRAS in  press, 
#                 arXiv:astro-ph/1603.06256
#
# 1 response variable (Y - galaxy class Seyfert - 1/AGN - 0)
# 2 explanatory variable (x1 - M200, x2 - cluster-centric distance)
#
# Data from: Trevisan, Mamon & Khosroshahi, 2017, MNRAS, 464, p.4593-4610
#            https://github.com/COINtoolbox/LOGIT_AGNs/tree/master/data

In [23]:
import numpy as np
import pandas as pd
import pystan 
import statsmodels.api as sm

# Data
path_to_data = 'https://raw.githubusercontent.com/astrobayes/BMAD/master/data/Section_10p8/Seyfert.csv'

# read data
data_frame = pd.read_csv(path_to_data)

x1 = data_frame['logM200']
x2 = data_frame['r_r200']

data = {}
data['Y'] = data_frame['bpt']
data['X'] = sm.add_constant(np.column_stack((x1,x2)))
data['K'] = data['X'].shape[1]
data['N'] = data['X'].shape[0]
data['gal'] = [0 if item == data_frame['zoo'][0] else 1 
                 for item in data_frame['zoo']]
data['P'] = 2

In [6]:
print data_frame

      bpt   logM200    r_r200 zoo
0       0 -0.257200 -0.960219   E
1       0 -0.239955 -0.957191   E
2       1 -1.217851 -0.860220   E
3       0 -0.387050 -0.805846   E
4       1 -0.418366 -0.855198   E
5       0 -1.143149 -0.975864   E
6       0 -1.143149 -1.035848   E
7       0 -0.072236 -0.964835   E
8       1 -0.770014 -0.757996   E
9       1  1.110179 -0.707352   E
10      1 -0.982879 -0.978683   E
11      0  0.001363 -0.931813   E
12      0 -0.036644 -0.841155   E
13      1 -0.450061 -0.975333   E
14      1 -0.032988 -0.723310   E
15      0  1.562362 -0.603055   E
16      1  1.562362 -0.616265   E
17      0 -1.209471 -0.859474   E
18      1 -0.719040 -0.775250   E
19      0 -0.719040 -0.997559   E
20      1 -0.345043 -0.512724   E
21      1 -0.002845 -0.408073   E
22      1 -0.002845 -0.752622   E
23      1  0.826370 -0.926728   E
24      0  0.639165 -0.777269   E
25      1 -0.809883 -0.928594   E
26      0 -0.494931 -0.960362   E
27      1 -0.858443 -0.736455   E
28      0 -0.5

In [ ]:
# Fit
stan_code="""
data{
    int<lower=0> N;                # number of data points
    int<lower=0> K;                # number of coefficients
    int<lower=0> P;                # number of populations
    matrix[N,K] X;                 # [logM200, galactocentric distance]
    int<lower=0, upper=1> Y[N];    # Seyfert 1/AGN 0
    int<lower=0, upper=1> gal[N];  # elliptical 0/spiral 1
}
parameters{
    matrix[K,P] beta;
    real<lower=0> sigma;
    real mu;
}
model{
    vector[N] pi;

    for (i in 1:N) {
        if (gal[i] == gal[1]) 
            pi[i] = dot_product(col(beta,1), X[i]);
        else 
            pi[i] = dot_product(col(beta,2), X[i]);
    }

    # shared hyperpriors
    sigma ~ gamma(0.001, 0.001);
    mu ~ normal(0, 100);

    # priors and likelihood
    for (i in 1:K) {
        for (j in 1:P) beta[i,j] ~ normal(mu, sigma);
    }

    Y ~ bernoulli_logit(pi);
}
"""

# Run mcmc
fit = pystan.stan(model_code=stan_code, data=data, iter=5000, chains=3,
                  warmup=3000, thin=10, n_jobs=3)

# Output
print(fit)
